import pandas as pd

In [1]:
import pandas as pd

In [2]:
barbar = pd.read_csv(r'C:\Users\Dell\Desktop\Attractions\Museum of Barbarism.csv', error_bad_lines=False);
goldentulip = pd.read_csv(r'C:\Users\Dell\Desktop\Hotel\Golden Tulip Nicosia Hotel and Casino.csv', error_bad_lines=False);
merit = pd.read_csv(r'C:\Users\Dell\Desktop\Hotel\Merit Lefkosa Hotel & Casino.csv', error_bad_lines=False);

tripadv_review = pd.concat([barbar, goldentulip, merit])
tripadv_review

,r_points,r_text,place
0,30,It's one of those strange things that some of ...,Museum of Barbarism
1,50,A place to show you the civilized human beings...,Museum of Barbarism
2,10,My father was the British RSM RAMC that was in...,Museum of Barbarism
3,10,This 'museum' is located in one of the busiest...,Museum of Barbarism
4,30,"As soon as you get in, it reverberates sadness...",Museum of Barbarism
...,...,...,...
43,10,This is a five-star hotel on the Turkish side ...,Merit Lefkosa Hotel & Casino
44,30,This hotel is booked for me when I travel to N...,Merit Lefkosa Hotel & Casino
45,40,a hotel with all mod cons and good service in ...,Merit Lefkosa Hotel & Casino
46,40,After my staying at Golden and 3 days in Karpa...,Merit Lefkosa Hotel & Casino


In [3]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [4]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

In [6]:
processed_reviews = tripadv_review['r_text'].map(preprocess)
processed_reviews

0     [strange, things, point, visiting, sites, horr...
1             [place, civilized, human, beings, europe]
2     [father, british, ramc, charge, bringing, bodi...
3     [museum, located, busiest, trendiest, streets,...
4     [soon, reverberates, sadness, especially, clot...
                            ...                        
43    [star, hotel, turkish, cyprus, beautiful, plac...
44    [hotel, booked, travel, nicosia, business, pur...
45    [hotel, cons, good, service, easy, walking, di...
46    [staying, golden, days, karpaz, area, came, ni...
47    [hotel, years, chain, great, disappointed, bus...
Name: r_text, Length: 120, dtype: object

In [7]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(processed_reviews)

# Create Corpus
texts = processed_reviews

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1)]]


In [14]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=50,
                                           update_every=1,
                                           chunksize=150,
                                           passes=2,
                                           alpha='auto',
                                           eta='auto',
                                           per_word_topics=True,
                                           iterations = 4,
                                           eval_every = None)

In [15]:
from pprint import pprint

In [16]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.023*"hotel" + 0.015*"room" + 0.013*"good" + 0.008*"staff" + '
  '0.007*"service" + 0.007*"breakfast" + 0.007*"casino" + 0.006*"rooms" + '
  '0.006*"clean" + 0.005*"stay"'),
 (1,
  '0.036*"hotel" + 0.025*"good" + 0.016*"staff" + 0.013*"breakfast" + '
  '0.012*"room" + 0.008*"stay" + 0.008*"food" + 0.007*"nice" + 0.007*"casino" '
  '+ 0.007*"clean"'),
 (2,
  '0.036*"hotel" + 0.013*"staff" + 0.012*"good" + 0.012*"breakfast" + '
  '0.008*"rooms" + 0.008*"room" + 0.008*"casino" + 0.007*"turkish" + '
  '0.007*"service" + 0.006*"nice"'),
 (3,
  '0.019*"hotel" + 0.016*"room" + 0.014*"good" + 0.013*"staff" + '
  '0.013*"breakfast" + 0.010*"casino" + 0.007*"clean" + 0.007*"rooms" + '
  '0.006*"stay" + 0.006*"turkish"'),
 (4,
  '0.017*"hotel" + 0.011*"staff" + 0.009*"room" + 0.009*"good" + '
  '0.008*"breakfast" + 0.007*"nice" + 0.007*"service" + 0.007*"casino" + '
  '0.007*"clean" + 0.006*"turkish"')]


In [17]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.012739  0.005017       1        1  25.228899
2     -0.004620 -0.011124       2        1  22.800411
3      0.003158  0.009568       3        1  20.879820
4      0.012804 -0.001128       4        1  15.561276
0      0.001398 -0.002333       5        1  15.529598, topic_info=            Term        Freq       Total Category  logprob  loglift
230         room   64.000000   64.000000  Default  30.0000  30.0000
214        hotel  150.000000  150.000000  Default  29.0000  29.0000
212         good   83.000000   83.000000  Default  28.0000  28.0000
102      turkish   27.000000   27.000000  Default  27.0000  27.0000
231      service   34.000000   34.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
19      location    2.438654   15.817892   Topic5  -5.8502  -0.0073
262  comfortable    2.572963   18.755461   Topic5  -5.7966  -0.1240
76          city    2.328483   14.182667   Topic5  -5.8965   0.0556
102      turkish    2.660729   27.832233   Topic5  -5.7631  -0.4852
268    excellent    2.339377   14.961819   Topic5  -5.8918   0.0068

[391 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
134       2  0.661359   accusing
1494      2  0.438768   addition
172       4  0.688414  agreement
1428      1  0.657889  allegedly
750       1  0.646112   allowing
...     ...       ...        ...
45        1  0.225723      years
45        2  0.225723      years
45        3  0.225723      years
45        4  0.225723      years
45        5  0.225723      years

[629 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 4, 5, 1])